# Creating the tardis_example database

This notebook provides a quick introduction in how to use Carsus.

## Prerequisites:
- Install carsus (preferably as a git repository)
- Install ChiantiPy (from source or pip, both versions work)
- Install chiantipy and set the environmentvariable `XUVTOP = /path/to/chianti/root`
- We highly recommend installing the `carsus-db` repository as it includes the files of several sources that cannot easily be downloaded.

## Imports and initialization:

Import all the essential ingesters and initialize the session.

In [1]:
import os
from carsus import init_db
from carsus.io.nist import (
        NISTWeightsCompIngester,
        NISTIonizationEnergiesIngester
        )
from carsus.io.kurucz import GFALLIngester
from carsus.io.chianti_ import ChiantiIngester
from carsus.io.zeta import KnoxLongZetaIngester
from carsus.io.output import AtomData


DB_ROOT = '/media/data/projects/tardis/carsus-db/'
session = init_db('sqlite:////tmp/carsus_example.db')
#session = init_db('sqlite:///' + os.path.join(DB_ROOT, 'carsus_example.db'))
storename = os.path.join(DB_ROOT, 'kurucz_chianti_H_He.h5')
storename = '/tmp/carsus_example.h5'
zeta_fname = os.path.join(DB_ROOT, 'zeta/knox_long_recombination_zeta.dat')
gfall_fname = os.path.join(DB_ROOT, 'gfall/gfall_old.dat')

 found PyQt4 widgets
 using PyQt4 widgets
Initializing the database at sqlite:////tmp/carsus_example.db


## Ingesting data


### Weights

In [ ]:
weightscomp_ingester = NISTWeightsCompIngester(session)
weightscomp_ingester.ingest()
session.commit()

Now we have all the weights ingested.

### Ionization Energies

In [ ]:
ioniz_energies_ingester = NISTIonizationEnergiesIngester(
            session,
            spectra="h-zn"
            )
ioniz_energies_ingester.ingest(
            ionization_energies=True,
            ground_levels=True
            )
session.commit()


### Ingesting kurucz levels and lines

In [ ]:
gfall_ingester = GFALLIngester(session, gfall_fname, ions='H-Zn')
gfall_ingester.ingest(levels=True, lines=True)
session.commit()

### Ingesting chianti levels and lines for H and He

In [ ]:
chianti_ingester = ChiantiIngester(session, ions='h-he')
chianti_ingester.ingest(levels=True, lines=True, collisions=False)
session.commit()

### Ingesting Zeta data

In [ ]:
zeta_ingester = KnoxLongZetaIngester(session, zeta_fname)
zeta_ingester.ingest()
session.commit()

## Create a tardis readable HDFStore

As we did in the Quickstart, we first create an `AtomData` instance. For this database we choose kurucz as the primary `DataSource` with only hydrogen and helium coming from chianti. We also specify `chianti_short_name` which contains the version of the chianti database that was ingested.

In [2]:
ad = AtomData(
        session,
        selected_atoms='H-Zn',
        chianti_ions="H; He",
        chianti_short_name='chianti_v8.0.6'
        )



### Writing the output

The database we want to create contains all data, except for the collision data.

In [3]:
ad.to_hdf(
        storename,
        store_atom_masses=True,
        store_ionization_energies=True,
        store_levels=True,
        store_lines=True,
        store_macro_atom=True,
        store_collisions=False,
        store_zeta_data=True
        )

/media/data/projects/tardis/carsus/carsus/io/output/tardis_.py:562: RuntimeWarning: divide by zero encountered in log10
  lines['loggf'] = np.log10(lines['gf'])


Signing AtomData with MD5 and UUID1


Comparison to old kurucz: 7 lines missing

You are done! Now you can use the created HDFStore to run TARDIS simulations.